In [7]:
# Pull stock data using Alpha Vantage

import requests
import json
from datetime import datetime
import pandas as pd
import AV_Errors

API_URL = "https://www.alphavantage.co/query"

API_KEY = "VK80229IJIZZ6BV3"

In [10]:
# Pull stock data using Alpha Vantage

def time_series(function, adj):
    time_series_string = ""

    if adj and function == "DAILY":
        time_series_string = "Time Series (Daily)"

    elif adj:
        series = " Adjusted Time Series"
        time_series_string = function[0] + function[1:].lower() + series

    elif function.upper() == "DAILY":
        time_series_string = "Time Series (Daily)"

    else:
        time_series_string = function[0] + function[1:].lower() + " Time Series"

    return time_series_string


def format_df(function, df, symbol, adj):
    if function == "DAILY" and adj:
        col_extra1 = ["adjusted close"]
        col_extra2 = ["dividend amount", "split coefficient"]
    elif adj:
        col_extra1 = ["adjusted close"]
        col_extra2 = ["dividend amount"]
    else:
        col_extra1 = []
        col_extra2 = []

    df.columns = ["open", "high", "low", "close"] + col_extra1 + ["volume"] + col_extra2
    df["meta"] = ""
    df.insert(0, "date", df.index)

    df = df.reindex(index=df.index[::-1])
    date = str(datetime.now())
    df["meta"][0] = [['src', 'Alpha Vantage'],
                     ['date', date],
                     ['symb', symbol],
                     ['function', function.lower() + ("adj" if adj else "")],
                     ['resolution', 'n/a'],
                     ['start', str(df.tail(1)["date"]).split()[0]],
                     ['end', str(df.head(1)["date"]).split()[0]],
                     ['length', str(len(df))]]

    return df


def get_historical(function, symbol, ip="default"):
    if function not in ["MONTHLY", "DAILY", "WEEKLY"]:
        raise Exception("Invalid value: '" + function + "' \n"
                        " Acceptable values are: 'MONTHLY', 'WEEKLY', " +
                        "'DAILY'")

    data = {"function": "TIME_SERIES_" + function.upper(),
            "symbol": symbol,
            "datatype": "json",
            "outputsize": "full",
            "apikey": API_KEY}

    data = AV_Errors.request_err_production(API_URL, data, ip)

    AV_Errors.df_key_err_production(data, symbol)

    df = pd.DataFrame.from_dict(data[time_series(function,
                                                 False)]).transpose()

    return format_df(function, df, symbol, False)


def get_historical_adjusted(function, symbol, ip="default"):
    if function not in ["MONTHLY", "DAILY", "WEEKLY"]:
        raise Exception("Invalid value: '" + function + "' \n"
                        " Acceptable values are: 'MONTHLY', 'WEEKLY', 'DAILY'")

    data = {"function": "TIME_SERIES_" + function.upper() + "_ADJUSTED",
            "symbol": symbol,
            "datatype": "json",
            "outputsize": "full",
            "apikey": API_KEY}

    data = AV_Errors.request_err_production(API_URL, data, ip)

    AV_Errors.df_key_err_production(data, symbol)

    df = pd.DataFrame.from_dict(data[time_series(function,
                                                 True)]).transpose()

    return format_df(function, df, symbol, True)


def get_sector_perf():
    data = {"function": "SECTOR",
            "apikey": API_KEY}

    data = AV_Errors.request_err_production(API_URL, data)

    key_lst = []
    val_lst = []

    for key in list(keys)[1:]:
        val_lst.append(data[key])
        key_lst.append(key.split(":")[1][1:])
    df = pd.DataFrame()
    df["sector period"] = key_lst

    real_est = []
    energy = []
    utilities = []
    financials = []
    staples = []
    health_care = []
    industrials = []
    materials = []
    IT = []
    consumer_discr = []
    x = []
    communic_serv = []

    for key in list(data.keys())[1:]:
        real_est.append(data[key]["Real Estate"] if "Real Estate" in data[key]
                        else "")
        energy.append(data[key]["Energy"])
        utilities.append(data[key]["Utilities"])
        financials.append(data[key]["Financials"])
        staples.append(data[key]["Consumer Staples"])
        health_care.append(data[key]["Health Care"])
        industrials.append(data[key]["Industrials"])
        materials.append(data[key]["Materials"])
        IT.append(data[key]["Information Technology"])
        consumer_discr.append(data[key]["Consumer Discretionary"])
        communic_serv.append(data[key]["Communication Services"])

    df["Real Estate"] = real_est
    df["Energy"] = energy
    df["Utilities"] = utilities
    df["Financials"] = financials
    df["Consumer Staples"] = staples
    df["Health Care"] = health_care
    df["Industrials"] = industrials
    df["Materials"] = materials
    df["Information Technology"] = IT
    df["Communication Services"] = consumer_discr
    df["Materials"] = communic_serv

    df = df.reindex(index=df.index[::-1])
    return df


'''
data = {"function": "TIME_SERIES_MONTHLY",
        "symbol": "MSFT",
        "datatype": "json",
        "outputsize": "full",
        "apikey": userInfo.key_gen()}

print("###################################################################")
response = requests.get(API_URL, data, proxies={
                        "http": "212.30.52.37:42939", "https": "212.30.52.37:42939"}, timeout=1)

print(response)
'''


'\ndata = {"function": "TIME_SERIES_MONTHLY",\n        "symbol": "MSFT",\n        "datatype": "json",\n        "outputsize": "full",\n        "apikey": userInfo.key_gen()}\n\nprint("###################################################################")\nresponse = requests.get(API_URL, data, proxies={\n                        "http": "212.30.52.37:42939", "https": "212.30.52.37:42939"}, timeout=1)\n\nprint(response)\n'

In [18]:
pip install sklearn

     |████████████████████████████████| 8.0 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 33.2 MB 5.6 MB/s eta 0:00:012
     |████████████████████████████████| 306 kB 4.7 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=109301d663e99b43e49e56e54a69bf017cb9657e778fabc9b4a0987ed758b908
  Stored in directory: /Users/j/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
#!/usr/bin/env python
# coding: utf-8

# # *get_stock_data*
# 
# Utility functions to download and retrieve stock data and then quickly retrieve it from the created database
# 
# 

# In[81]:


# Packages

# Package Imports
import time
import os
import pandas as pd
import numpy as np
import pandas_datareader as pddr # Returns historical stock information: pip install pandas-datareader
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_finance import candlestick_ohlc # Extends matplot for financial plotting: pip install mpl-finance 
from datetime import datetime
from sklearn import preprocessing # used for normalizing data




ROOT = '/Users/j/Desktop/research/'


# In[86]:


def return_differences(graph, axis=0):
    # returns a list of the differences between each point in the graph
    l = []
    g = sorted(graph)
    for i in range(len(g)-1):
        l.append(g[i+1][axis] - g[i][axis])
    
    return l
        


# In[87]:


def log_changes(y_values, start_price=-1):
    """"Returns the log changes for a list of y values.

    Args:
        y_values (List): The list of ordered y values from the fractal graph.
        start_price (int): Useful if stock starts at certain price.

    Returns:
        DataDrame: Returns a dataframe with the log returns.
    """
    if start_price == -1:
        start_price = abs(min(y_values)) + 1
    
    fixed = [x+start_price for x in y_values]
    df = pd.DataFrame(fixed, columns=['price'])
    df['pct_change'] = df.price.pct_change()
    df['log_ret'] = np.log(df.price) - np.log(df.price.shift(1))
    return df['log_ret']


# In[88]:


def plot_data_list(data, column='Normalized Close', save=False):
    fig, ax = plt.subplots() # Needs to go before everything else
    tickers = []
    #values = []
    for key in data.keys():
        tickers.append(key)
        #values.append(data[key][column])
        stock = data[key]
        ax.plot(stock['Date'], list(stock[column]))
    
    
    plt.legend(tickers,loc='best')
    plt.xlabel('Date')
    plt.ylabel(column)
    plt.title(tickers[0] + ', ' + ', '.join(tickers[1:])+ ' ' + column)
    
    # round to nearest years.
    #datemin = np.datetime64(data['date'][0], 'Y')
    #datemax = np.datetime64(data['date'][-1], 'Y') + np.timedelta64(1, 'Y')
    #ax.set_xlim(datemin, datemax)

    # format the coords message box
    #ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')

    
    plt.grid(color='k', linestyle='-', linewidth=0.1)
    plt.tick_params(axis='x', which='major', labelsize=10)
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    plt.xticks(rotation=45)
    plt.gcf().set_size_inches(30,4)
    #matplotlib.rcParams['figure.dpi'] = 200
    
    
    


# In[89]:


def add_normalized_close(df):
    # Create x, where x the 'scores' column's values as floats
    x = df[['Adj Close']].values.astype(float)

    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)

    # Run the normalizer on the dataframe
    df['Normalized Close'] = x_scaled
    
    return df

def get_existing_tickers():
    lst = []
    basepath = ROOT + 'Daily'
    for entry in os.listdir(basepath):
        if os.path.isfile(os.path.join(basepath, entry)):
            lst.append(entry)
            
    return lst
    
                 
            
def resource_data_list(lst, start='2010-1-1', end='2020-1-1',overwrite=False):
    data = {}
    
    existing_tickers = get_existing_tickers()
    
    for ticker in lst:
        try:
            if ticker + '.csv' in existing_tickers and not(overwrite):
                data[ticker] = pd.read_csv(ROOT + 'Daily/' + ticker + '.csv')

            else:
                stock = pddr.DataReader(ticker, 
                               start=start, 
                               end = end,
                               data_source='yahoo')

                stock = add_normalized_close(stock)
                data[ticker] = stock

                stock.to_csv(ROOT + 'Daily/' + ticker + '.csv' )
        except Exception as e:
            print("Error: ", ticker, e)
            
            
        
    return data
    



In [ ]:
data=resource_data_list([])

In [11]:
data = get_historical("MONTHLY", "AAPL")

In [12]:
data

,date,open,high,low,close,volume,meta
1999-12-31,1999-12-31,101.0000,118.0000,91.0600,102.8100,84091200,"[[src, Alpha Vantage], [date, 2021-12-19 12:10..."
2000-01-31,2000-01-31,104.8700,121.5000,86.5000,103.7500,112099800,
2000-02-29,2000-02-29,104.0000,119.9400,97.0000,114.6200,65355200,
2000-03-31,2000-03-31,118.5600,150.3800,114.0000,135.8100,77663900,
2000-04-28,2000-04-28,135.5000,139.5000,104.8700,124.0600,77342900,
...,...,...,...,...,...,...,...
2021-08-31,2021-08-31,146.3600,153.4900,144.5000,151.8300,1462773381,
2021-09-30,2021-09-30,152.8300,157.2600,141.2700,141.5000,1797948421,
2021-10-29,2021-10-29,141.9000,153.1650,138.2700,149.8000,1565079040,
2021-11-30,2021-11-30,148.9850,165.7000,147.4800,165.3000,1688864233,
